**Modelo de Clasificación con ResNet50 - Ejecutado en Kaggle**

In [ ]:
# CELDA 1: Verificar dataset
import os

# Ver estructura
print("📁 Contenido del dataset:")
!ls -lh /kaggle/input/food-101/food-101/food-101/

print("\n📊 Carpetas principales:")
!ls /kaggle/input/food-101/food-101/food-101/images/


In [ ]:
# ═══════════════════════════════════════════════════════════
# CELDA 2: Imports y configuración
# ═══════════════════════════════════════════════════════════

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import os
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

# Verificar GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔥 Dispositivo: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Configuración
BATCH_SIZE = 64  # Ajustar según VRAM
NUM_EPOCHS = 30
LEARNING_RATE = 0.005
NUM_WORKERS = 2


In [ ]:
# ═══════════════════════════════════════════════════════════
# CELDA 3: Split 70/15/15
# ═══════════════════════════════════════════════════════════

import glob
from sklearn.model_selection import train_test_split

class Food101Dataset(Dataset):
    """Dataset para Food-101"""
    
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


# ═══════════════════════════════════════════════════════════
# Preparar datos con split 70/15/15
# ═══════════════════════════════════════════════════════════

print("📂 Preparando dataset Food-101...\n")

DATA_PATH = '/kaggle/input/food-101/food-101/food-101'
IMAGES_PATH = os.path.join(DATA_PATH, 'images')

# Obtener clases
classes = sorted([d for d in os.listdir(IMAGES_PATH) 
                 if os.path.isdir(os.path.join(IMAGES_PATH, d))])

print(f"✓ Clases encontradas: {len(classes)}")

# Mapeo clase -> índice
class_to_idx = {cls: idx for idx, cls in enumerate(classes)}

# Recolectar paths e labels
all_image_paths = []
all_labels = []

print("\n📊 Escaneando imágenes...")
for cls in tqdm(classes):
    cls_path = os.path.join(IMAGES_PATH, cls)
    images = glob.glob(os.path.join(cls_path, '*.jpg'))
    
    all_image_paths.extend(images)
    all_labels.extend([class_to_idx[cls]] * len(images))

print(f"\n✓ Total imágenes: {len(all_image_paths)}")

# ═══════════════════════════════════════════════════════════
# Split 70/15/15: Train/Val/Test
# ═══════════════════════════════════════════════════════════

print("\n🔀 Dividiendo dataset (70% train, 15% val, 15% test)...")

# Primero: 70% train, 30% temp (val+test)
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    all_image_paths, 
    all_labels,
    test_size=0.30,      # 30% para val+test
    random_state=42,
    stratify=all_labels
)

# Segundo: dividir temp en 50/50 (15% val, 15% test)
val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths,
    temp_labels,
    test_size=0.50,      # 50% de 30% = 15% total
    random_state=42,
    stratify=temp_labels
)

# Verificar proporciones
total = len(all_image_paths)
print(f"\n📊 Distribución final:")
print(f"   Train: {len(train_paths):,} ({100*len(train_paths)/total:.1f}%)")
print(f"   Val:   {len(val_paths):,} ({100*len(val_paths)/total:.1f}%)")
print(f"   Test:  {len(test_paths):,} ({100*len(test_paths)/total:.1f}%)")

# Guardar clases
with open('food101_classes.json', 'w') as f:
    json.dump(classes, f, indent=4)

print("\n✓ Clases guardadas en food101_classes.json")


In [ ]:
# ═══════════════════════════════════════════════════════════
# CELDA 4: Transformaciones (Data Augmentation)
# ═══════════════════════════════════════════════════════════

# Normalización de ImageNet (IMPORTANTE para Transfer Learning)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])

# Train: Con augmentation
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(25),  # ← Aumentar de 20 a 25
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),  # ← Más agresivo
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # ← NUEVO: pequeños shifts
    transforms.RandomGrayscale(p=0.1),  # ← NUEVO: ocasionalmente B/N
    transforms.ToTensor(),
    normalize,
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.15))  # ← NUEVO: random erasing
])

# Test: Sin augmentation
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

print("✓ Transformaciones definidas")


In [ ]:
# ═══════════════════════════════════════════════════════════
# CELDA 5 ACTUALIZADA: Crear Train/Val/Test Loaders
# ═══════════════════════════════════════════════════════════

print("📊 Creando datasets y loaders...\n")

# Crear 3 datasets
train_dataset = Food101Dataset(train_paths, train_labels, transform=train_transform)
val_dataset = Food101Dataset(val_paths, val_labels, transform=test_transform)
test_dataset = Food101Dataset(test_paths, test_labels, transform=test_transform)

print(f"✓ Train: {len(train_dataset):,} muestras")
print(f"✓ Val:   {len(val_dataset):,} muestras")
print(f"✓ Test:  {len(test_dataset):,} muestras")

# DataLoaders
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

print(f"\n✓ Train batches: {len(train_loader)}")
print(f"✓ Val batches:   {len(val_loader)}")
print(f"✓ Test batches:  {len(test_loader)}")

# Verificar un batch
images, labels = next(iter(train_loader))
print(f"\n✓ Batch shape: {images.shape}")
print(f"✓ Primer label: {labels[0].item()} -> {classes[labels[0].item()]}")


In [ ]:
# ═══════════════════════════════════════════════════════════
# CELDA 6: Modelo ResNet50
# ═══════════════════════════════════════════════════════════

class ResNet50Food101(nn.Module):
    """ResNet50 preentrenado para Food-101"""
    
    def __init__(self, num_classes=101, freeze_base=True):
        super(ResNet50Food101, self).__init__()
        
        # Cargar ResNet50 preentrenado
        self.resnet = models.resnet50(pretrained=True)
        
        # Congelar capas base
        if freeze_base:
            for param in self.resnet.parameters():
                param.requires_grad = False
        
        # Reemplazar última capa
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Dropout(0.6),
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        return self.resnet(x)
    
    def unfreeze(self):
        """Descongela todas las capas para fine-tuning"""
        for param in self.resnet.parameters():
            param.requires_grad = True


# Crear modelo
modelo = ResNet50Food101(num_classes=101, freeze_base=True).to(device)

# Contar parámetros
total_params = sum(p.numel() for p in modelo.parameters())
trainable_params = sum(p.numel() for p in modelo.parameters() if p.requires_grad)

print(f"\n📊 Modelo ResNet50:")
print(f"   Parámetros totales: {total_params:,}")
print(f"   Parámetros entrenables: {trainable_params:,}")
print(f"   Congelados: {total_params - trainable_params:,}")


In [ ]:
# ═══════════════════════════════════════════════════════════
# CELDA 7: Funciones de entrenamiento y evaluación
# ═══════════════════════════════════════════════════════════
def mixup_data(x, y, alpha=0.2):
    """Mixup augmentation"""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


def train_epoch(modelo, loader, criterion, optimizer, device, use_mixup=True):
    """Entrena una época con Mixup opcional"""
    modelo.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc='Training')
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        # Aplicar Mixup
        if use_mixup:
            images, labels_a, labels_b, lam = mixup_data(images, labels, alpha=0.2)
        
        optimizer.zero_grad()
        outputs = modelo(images)
        
        # Loss con Mixup
        if use_mixup:
            loss = mixup_criterion(criterion, outputs, labels_a, labels_b, lam)
        else:
            loss = criterion(outputs, labels)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(modelo.parameters(), max_norm=1.0)
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        
        # Accuracy: con Mixup usamos la etiqueta dominante (lam)
        if use_mixup:
            correct += (lam * (predicted == labels_a).float().sum().item() + 
                       (1 - lam) * (predicted == labels_b).float().sum().item())
        else:
            correct += (predicted == labels).sum().item()
        
        pbar.set_postfix({
            'loss': f'{running_loss/(pbar.n+1):.4f}',
            'acc': f'{100*correct/total:.2f}%'
        })
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = 100 * correct / total
    
    return epoch_loss, epoch_acc


def evaluate(modelo, loader, criterion, device):
    """Evalúa el modelo (sin Mixup)"""
    modelo.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        pbar = tqdm(loader, desc='Evaluating')
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            
            outputs = modelo(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            pbar.set_postfix({
                'loss': f'{running_loss/(pbar.n+1):.4f}',
                'acc': f'{100*correct/total:.2f}%'
            })
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = 100 * correct / total
    
    return epoch_loss, epoch_acc

# Al final de CELDA 7 (opcional)
print("\n✅ Funciones definidas correctamente:")
print(f"   - mixup_data()")
print(f"   - mixup_criterion()")
print(f"   - train_epoch() con Mixup")
print(f"   - evaluate()")


In [ ]:
# ═══════════════════════════════════════════════════════════
# CELDA 8 MEJORADA: FASE 1 con Early Stopping
# ═══════════════════════════════════════════════════════════

print("="*70)
print("🚀 FASE 1: Entrenando capas FC (base congelada)")
print("="*70)

# Loss y Optimizer
criterion = nn.CrossEntropyLoss(label_smoothing=0.2)  # ← Era 0.1, aumentar
optimizer = optim.AdamW(modelo.resnet.fc.parameters(), lr=LEARNING_RATE, weight_decay=1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)

# Historial
history = {
    'train_loss': [], 'train_acc': [],
    'val_loss': [], 'val_acc': []
}

# ✅ EARLY STOPPING
mejor_acc = 0.0
FASE1_EPOCHS = 50  # ← Épocas máximas aumentadas
patience = 10       # ← Parar si no mejora en 7 épocas
epocas_sin_mejora = 0

for epoch in range(FASE1_EPOCHS):
    print(f"\n{'='*70}")
    print(f"Época {epoch+1}/{FASE1_EPOCHS} [FASE 1]")
    print(f"{'='*70}")
    
    # Entrenar
    train_loss, train_acc = train_epoch(modelo, train_loader, criterion, optimizer, device)
    
    # Evaluar
    val_loss, val_acc = evaluate(modelo, val_loader, criterion, device)
    
    # Scheduler
    scheduler.step()
    
    # Guardar historial
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Resumen
    print(f"\n📊 Resultados Época {epoch+1}:")
    print(f"   Train: Loss={train_loss:.4f}, Acc={train_acc:.2f}%")
    print(f"   Val:   Loss={val_loss:.4f}, Acc={val_acc:.2f}%")
    print(f"   LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    # ✅ Verificar mejora
    if val_acc > mejor_acc:
        mejor_acc = val_acc
        epocas_sin_mejora = 0  # Reset contador
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': modelo.state_dict(),
            'val_acc': val_acc,
            'classes': classes
        }, 'resnet50_food101_fase1.pth')
        print(f"   ✅ Mejor modelo guardado (Val Acc: {val_acc:.2f}%)")
    else:
        epocas_sin_mejora += 1
        print(f"   ⏸️  Sin mejora ({epocas_sin_mejora}/{patience} épocas)")
    
    # ✅ EARLY STOPPING: Detener si no mejora
    if epocas_sin_mejora >= patience:
        print(f"\n🛑 EARLY STOPPING: No mejora en {patience} épocas")
        print(f"   Mejor Val Acc: {mejor_acc:.2f}% (Época {epoch - patience + 1})")
        break

print(f"\n✅ FASE 1 COMPLETADA. Mejor Val Acc: {mejor_acc:.2f}%")


In [ ]:
# ═══════════════════════════════════════════════════════════
# CELDA 9: FASE 2 con Early Stopping
# ═══════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🚀 FASE 2: Fine-tuning completo")
print("="*70)

# Descongelar modelo
modelo.unfreeze()

FASE2_EPOCHS = 56  # ← Épocas máximas aumentadas

# Nuevo optimizer
optimizer = optim.AdamW(modelo.parameters(), lr=0.0001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=FASE2_EPOCHS)  # Mejor que ReduceLROnPlateau

# ✅ EARLY STOPPING
patience = 10      # ← Parar si no mejora en 10 épocas
epocas_sin_mejora = 0

for epoch in range(FASE2_EPOCHS):
    print(f"\n{'='*70}")
    print(f"Época {epoch+1}/{FASE2_EPOCHS} [FASE 2]")
    print(f"{'='*70}")
    
    # Entrenar
    train_loss, train_acc = train_epoch(modelo, train_loader, criterion, optimizer, device)
    
    # Evaluar
    val_loss, val_acc = evaluate(modelo, val_loader, criterion, device)
    
    # Scheduler
    scheduler.step()
    
    # Guardar historial
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Resumen
    print(f"\n📊 Resultados Época {FASE1_EPOCHS + epoch + 1}:")  # ✅ Época total correcta
    print(f"   Train: Loss={train_loss:.4f}, Acc={train_acc:.2f}%")
    print(f"   Val:   Loss={val_loss:.4f}, Acc={val_acc:.2f}%")
    print(f"   LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    # ✅ Verificar mejora
    if val_acc > mejor_acc:
        mejor_acc = val_acc
        epocas_sin_mejora = 0  # Reset contador
        
        torch.save({
            'epoch': epoch + FASE1_EPOCHS,
            'model_state_dict': modelo.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'classes': classes
        }, 'resnet50_food101_best.pth')
        print(f"   ✅ Mejor modelo guardado (Val Acc: {val_acc:.2f}%)")
    else:
        epocas_sin_mejora += 1
        print(f"   ⏸️  Sin mejora ({epocas_sin_mejora}/{patience} épocas)")

    # GUARDA HISTORIAL DESPUÉS DE CADA ÉPOCA ⬇️
    with open('training_history.json', 'w') as f:
        json.dump(history, f, indent=4)
        
    # ✅ EARLY STOPPING
    if epocas_sin_mejora >= patience:
        print(f"\n🛑 EARLY STOPPING: No mejora en {patience} épocas")
        print(f"   Mejor Val Acc: {mejor_acc:.2f}%")
        break

# Guardar modelo final
torch.save({
    'model_state_dict': modelo.state_dict(),
    'val_acc': val_acc,
    'classes': classes
}, 'resnet50_food101_final.pth')

# Guardar historial
with open('training_history.json', 'w') as f:
    json.dump(history, f, indent=4)

print(f"\n✅ ENTRENAMIENTO COMPLETADO")
print(f"   Mejor Val Accuracy: {mejor_acc:.2f}%")


In [ ]:
# ═══════════════════════════════════════════════════════════
# CELDA 10: Visualizar curvas
# ═══════════════════════════════════════════════════════════

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_loss'], label='Val Loss', linewidth=2)  # ✅ Cambiar a 'val_loss'
axes[0].axvline(x=FASE1_EPOCHS, color='red', linestyle='--', label='Fase 2 inicio')
axes[0].set_xlabel('Época', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Loss durante Entrenamiento', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history['train_acc'], label='Train Acc', linewidth=2)
axes[1].plot(history['val_acc'], label='Val Acc', linewidth=2)    # ✅ Cambiar a 'val_acc'
axes[1].axvline(x=FASE1_EPOCHS, color='red', linestyle='--', label='Fase 2 inicio')
axes[1].axhline(y=mejor_acc, color='green', linestyle='--', label=f'Mejor: {mejor_acc:.2f}%')
axes[1].set_xlabel('Época', fontsize=12)
axes[1].set_ylabel('Accuracy (%)', fontsize=12)
axes[1].set_title('Accuracy durante Entrenamiento', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Gráficas guardadas")


In [ ]:
# ═══════════════════════════════════════════════════════════
# CELDA 11: Evaluación final en TEST
# ═══════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🎯 EVALUACIÓN FINAL EN TEST SET")
print("="*70)

# Cargar mejor modelo
checkpoint = torch.load('resnet50_food101_best.pth')
modelo.load_state_dict(checkpoint['model_state_dict'])

# Evaluar en test
test_loss, test_acc = evaluate(modelo, test_loader, criterion, device)

print(f"\n📊 RESULTADOS FINALES:")
print(f"   Mejor Val Acc:  {mejor_acc:.2f}%")
print(f"   Test Acc:       {test_acc:.2f}%")
print(f"   Diferencia:     {abs(test_acc - mejor_acc):.2f}%")

if abs(test_acc - mejor_acc) < 3:
    print("\n✅ Modelo generaliza bien (diferencia < 3%)")
else:
    print("\n⚠️ Posible overfitting (diferencia > 3%)")
